<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 理解 Embedding 层与 Linear 层的区别


- 在 PyTorch 中，嵌入层（Embedding layers）的功能与执行矩阵乘法的线性层（Linear layers）相同；我们使用嵌入层的原因是为了计算效率。
- 我们将通过 PyTorch 的代码示例，一步步观察这种关系。


In [1]:
import torch

print("PyTorch version:", torch.__version__)

PyTorch version: 2.3.1


<br>
&nbsp;

## 使用 nn.Embedding


In [2]:
# Suppose we have the following 3 training examples,
# which may represent token IDs in a LLM context
idx = torch.tensor([2, 3, 1])

# The number of rows in the embedding matrix can be determined
# by obtaining the largest token ID + 1.
# If the highest token ID is 3, then we want 4 rows, for the possible
# token IDs 0, 1, 2, 3
num_idx = max(idx)+1

# The desired embedding dimension is a hyperparameter
out_dim = 5

- 让我们实现一个简单的嵌入层：


In [3]:
# We use the random seed for reproducibility since
# weights in the embedding layer are initialized with
# small random values
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

我们可以选择查看嵌入层的权重：


In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 然后，我们可以使用嵌入层获取训练样本 ID 为 1 的向量表示：


In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面是对嵌入层内部运作的可视化示意：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 类似地，我们可以使用嵌入层获取训练样本 ID 为 2 的向量表示：


In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">

- 现在，让我们将之前定义的所有训练样本进行向量化：


In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- Under the hood, it's still the same look-up concept:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

<br>
&nbsp;

## 使用 nn.Linear


- 现在，我们将演示上述嵌入层在 PyTorch 中在独热编码表示（one-hot encoded representation）上所做的操作，与 `nn.Linear` 层完全相同。
- 首先，让我们将 token ID 转换为独热编码表示：


In [8]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，我们初始化一个 `Linear` 层，它执行矩阵乘法 $X W^\top$：


In [9]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 注意，PyTorch 中的线性层同样会用小的随机权重进行初始化；为了能够直接与上面的 `Embedding` 层进行比较，我们需要使用相同的小随机权重，这就是我们在这里重新赋值的原因：


In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

- 现在我们可以在输入的独热编码表示上使用线性层：


In [11]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

正如我们所看到的，这与使用嵌入层时得到的结果完全相同：


In [12]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层，对于第一个训练样本的 token ID，发生的计算如下：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 对于第二个训练样本的 token ID，计算如下：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每一行独热编码中只有一个索引为 1，其余都是 0（这是设计如此），因此这种矩阵乘法本质上相当于对独热元素的查找。
- 在独热编码上使用矩阵乘法的这种方式等效于嵌入层的查找，但如果我们处理大型嵌入矩阵时可能效率低下，因为会有大量对 0 的无效乘法。
